In [418]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [419]:
import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

import read_trace
import cuda_timeline
from avgblkmodel import *
import cke
from df_util import *
#from model_cke import *

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# gpu info

In [420]:
gtx950 = DeviceInfo()
gtx950.sm_num = 6
gtx950.sharedmem_per_sm = 49152
gtx950.reg_per_sm = 65536
gtx950.maxthreads_per_sm = 2048

# 2 stream info

In [421]:
# 10M for mem_mem : where the h2d between streams are overlapped
trace_file = 'trace_10M_s1.csv'
trace_file_2cke = 'trace_h2d_h2d_ovlp.csv'

df_trace = read_trace.trace2dataframe(trace_file) # read the trace to the dataframe
df_trace_2cke = read_trace.trace2dataframe(trace_file_2cke)

In [422]:
#df_trace

In [423]:
#cuda_timeline.plot_trace(df_trace)

In [424]:
#df_trace_2cke

In [425]:
#cuda_timeline.plot_trace(df_trace_2cke)

# 1cke - read trace and reset the timeline

In [426]:
df_single_stream = read_trace.get_timing(df_trace)

In [427]:
#df_single_stream

In [428]:
df_s1 = read_trace.reset_starting(df_single_stream)

In [429]:
#df_s1

### 2cke case

In [430]:
df_2stream = read_trace.get_timing(df_trace_2cke)

In [431]:
# df_2stream

In [432]:
tot_runtime = read_trace.getTotalRuntime(df_2stream)
print tot_runtime

37.784463


# 2 cke

In [433]:
stream_num = 2

# find when to start the stream and update the starting pos for the trace
H2D_H2D_OVLP_TH = 3.158431

df_cke_list = cke.init_trace_list(df_s1, stream_num = stream_num, h2d_ovlp_th = H2D_H2D_OVLP_TH)

In [434]:
#df_cke_list[0]

In [435]:
#df_cke_list[1]

### sort

In [436]:
df_all_api = cke.init_sort_api_with_extra_cols(df_cke_list)

In [437]:
#df_all_api

In [438]:
#df_all_api.loc[df_all_api.stream_id == 0]

In [439]:
# print df_all_api.iloc[0]
# print df_all_api.iloc[1]

In [440]:
#df_all_api

In [441]:
# pick the 1st sleep call and wake up
r1 = cke.pick_first_in_sleep(df_all_api)
df_all_api = SetWake(df_all_api, r1)

# pick another in the sleep mode, if it is from the same stream, there is non ovlp
r2 = cke.pick_first_in_sleep(df_all_api)
df_all_api = SetWake(df_all_api, r2)
#print('r2 {}'.format(r2))

# check concurrency
conc = cke.check_cc(df_all_api, r1, r2)

# concurrency
if conc == True:
    df_all_api = cke.update_before_conc(df_all_api, r1, r2)
    # set r2 to wake
    df_all_api = SetWake(df_all_api, r2)
    # current concurrency
    cc = 2.0
    # predict with concurrency for rows 0 and 1
    df_all_api = cke.Predict_end(df_all_api, [r1, r2], ways = cc)
    # get the time range from wake api, to check the next concurrent api
    rangeT = cke.Get_next_range(df_all_api)
    #print rangeT
    
    extra_conc = cke.check_cc_by_time(df_all_api, rangeT) # check whether there is conc during the rangeT
    
    if extra_conc == 0: # update timing using the pred_end
        df_all_api = cke.Update_with_pred_end(df_all_api, rangeT, ways = cc)
        # check if any api is done, and update the timing for the other apis in that stream
        df_all_api = cke.UpdateStreamTime(df_all_api)

In [442]:
df_all_api

,start,end,api_type,size_kb,stream_id,status,bw,bytes_done,bytes_left,current_pos,time_left,pred_end
0,0.000000,9.715199,h2d,38146.973,0.0,done,5926.373991,38146.97300,0.00000,9.715199,0.0,9.715199
4,3.158431,9.595246,h2d,38146.973,1.0,wake,5926.373991,19428.92967,18718.04333,9.715199,0.0,0.000000
1,9.716415,16.146414,h2d,38146.973,0.0,sleep,5932.656133,0.00000,38146.97300,0.000000,0.0,0.000000
5,9.596462,16.026461,h2d,38146.973,1.0,sleep,5932.656133,0.00000,38146.97300,0.000000,0.0,0.000000
2,16.157902,17.434520,kern,0.000,0.0,sleep,0.000000,0.00000,0.00000,0.000000,0.0,0.000000
3,17.441304,23.710950,d2h,38146.973,0.0,sleep,6084.390251,0.00000,38146.97300,0.000000,0.0,0.000000
6,16.037949,17.314567,kern,0.000,1.0,sleep,0.000000,0.00000,0.00000,0.000000,0.0,0.000000
7,17.321351,23.590997,d2h,38146.973,1.0,sleep,6084.390251,0.00000,38146.97300,0.000000,0.0,0.000000


In [443]:
## select the next api/call to wake it up

# pick another in the sleep mode, if it is from the same stream, there is non ovlp
r3 = cke.pick_first_in_sleep(df_all_api)
df_all_api = SetWake(df_all_api, r3)
#print r3

# the current_pos is ahead of coming call start
df_all_api = cke.StartNext(df_all_api, [r2, r3])

# check the concurrency, and predict accordingly
# to-do

1


In [444]:
df_all_api

,start,end,api_type,size_kb,stream_id,status,bw,bytes_done,bytes_left,current_pos,time_left,pred_end
0,0.000000,9.715199,h2d,38146.973,0.0,done,5926.373991,38146.97300,0.00000,9.715199,0.0,9.715199
4,3.158431,9.595246,h2d,38146.973,1.0,wake,5926.373991,19436.13614,18710.83686,9.716415,0.0,0.000000
1,9.716415,16.146414,h2d,38146.973,0.0,wake,5932.656133,0.00000,38146.97300,0.000000,0.0,0.000000
5,9.596462,16.026461,h2d,38146.973,1.0,sleep,5932.656133,0.00000,38146.97300,0.000000,0.0,0.000000
2,16.157902,17.434520,kern,0.000,0.0,sleep,0.000000,0.00000,0.00000,0.000000,0.0,0.000000
3,17.441304,23.710950,d2h,38146.973,0.0,sleep,6084.390251,0.00000,38146.97300,0.000000,0.0,0.000000
6,16.037949,17.314567,kern,0.000,1.0,sleep,0.000000,0.00000,0.00000,0.000000,0.0,0.000000
7,17.321351,23.590997,d2h,38146.973,1.0,sleep,6084.390251,0.00000,38146.97300,0.000000,0.0,0.000000


In [445]:
#df_all_api.loc[df_all_api.stream_id == 0]

In [446]:
#df_all_api.loc[df_all_api.stream_id == 1]

In [ ]:
#
# run above
#

# 3cke

# plot all the stream timeline

In [ ]:
cuda_timeline.plot_cke_list(df_cke_list, savefig=True)

In [ ]:
cuda_timeline.plot_cke_list(df_cke_list[0:2])

In [ ]:
tot_runtime = read_trace.getTotalRuntime(df_cke_list[0:2])
print tot_runtime